In [1]:
#Importing all important libraries which will be in use
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv')

In [3]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
df.columns = ['Target','Id', 'Time', 'Query', 'Name', 'Tweet']

In [5]:
df

,Target,Id,Time,Query,Name,Tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [6]:
df['Target'].value_counts()

4    800000
0    799999
Name: Target, dtype: int64

In [7]:
df.isnull().sum()

Target    0
Id        0
Time      0
Query     0
Name      0
Tweet     0
dtype: int64

In [8]:
tweets = df['Tweet']

In [9]:
tweets[100]

'@mangaaa I hope they will increase the capacity fast, yesterday was such a pain. Got the fail whale +15 times in 2 hours.... '

## Normalizing the text data first

In [10]:
import regex as re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

In [11]:
lemm = WordNetLemmatizer()

In [ ]:
corpus = []
for i in range(len(df)):
    tweet = re.sub('[^a-zA-Z]', ' ' , tweets[i])
    tweet = tweet.lower()
    tweet = tweet.split()
    tweet = [lemm.lemmatize(word) for word in tweet if word not in set(stopwords.words('english')) ]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

In [ ]:
corpus

In [2]:
from keras.preprocessing.text import one_hot

## Doing one hot representation

In [13]:
vocab_size = 10000

In [ ]:
one_hot_rep = [one_hot(word,vocab_size) for word in corpus]

## Padding

In [15]:
sent_len = 20

In [3]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
embedded_docs = pad_sequences(one_hot_rep, padding='pre', max_len=sent_len )

In [9]:
# Importing libraries which helps in making Bidirectional model
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import Dense

## Applying Embedding and Bidirectional layer

In [ ]:
dim = 100
model = Sequential()
model.add(Embedding(vocab_size,dim,input_length = sent_len))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
X = np.array(embedded_docs)
y = np.array(df['Target'])

## Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.43, random_state=42)

## Fitting the model

In [ ]:
model.fit(X_trin,y_train, validation_data=(X_test,y_test),epochs=100, batch_size=64)

In [ ]:
# Predicting y_test values
y_pred = model.predict(X_test)

# Performance Matrix

In [17]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report 

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
classification_report(y_pred,y_test)